In [ ]:
from IPython.display import display, DisplayHandle
from ipywidgets import interact, interactive, fixed, interact_manual

from PIL import Image
from PIL import ImageDraw
from PIL import ImageColor

import numpy as np
from numpy import matrix as M
from math import sin, cos, pi
import time

In [ ]:
# constants
WIDTH, HEIGHT = 300, 300

In [ ]:
i = Image.new("RGB", (WIDTH, HEIGHT))
draw = ImageDraw.Draw(i)

In [ ]:
draw.polygon([(150,150), (200,200), (200,150)], fill=(255,255,0))
display(i)

## 3D Experiment 1: Moving the Triangle

In [ ]:
# copied and adapted from Rotating Clock notebook
def translate(tx, ty, tz, p=None):
    T = M([[1, 0, 0, tx],
           [0, 1, 0, ty],
           [0, 0, 1, tz],
           [0, 0, 0,  1]])
    if p is None:
        return T
    else:
        p = list(p)
        p.append(1)
        p = T @ p
        x = p.tolist()[0][0]
        y = p.tolist()[0][1]
        z = p.tolist()[0][2]
        return((x,y,z))

In [ ]:
def rotate(axis, angle, p=None):
    angle = (angle / 180) * pi
    if axis == 'x':
        T = M([[1, 0, 0, 0],
           [0, cos(angle), -sin(angle), 0],
           [0, sin(angle), cos(angle), 0],
           [0, 0, 0,  1]])
    if axis == 'y':
        assert False
    if axis == 'z':
        assert False
    if p is None:
        return T
    else:
        p = list(p)
        p.append(1)
        p = T @ p
        x = p.tolist()[0][0]
        y = p.tolist()[0][1]
        z = p.tolist()[0][2]
        return((x,y,z))

In [ ]:
def project_ortho(p=None):
    T = M([[1, 0, 0,  0],
           [0, 1, 0,  0],
           [0, 0, 0,  0],
           [0, 0, 0,  1]])
    if p is None:
        return T
    else:
        p = list(p)
        p.append(1)
        p = T @ p
        x = p.tolist()[0][0]
        y = p.tolist()[0][1]
        #z = p.tolist()[0][2]
        return((x,y))

In [ ]:
def project_perspective(d=1, p=None):
    T = M([[1, 0, 0,  0],
           [0, 1, 0,  0],
           [0, 0, 1,  0],
           [0, 0, -1/d,  0]])
    if p is None:
        return T
    else:
        p = list(p)
        p.append(1)
        p = T @ p
        #print(p)
        w = p.tolist()[0][3]
        x = p.tolist()[0][0] / (w+0.0001) # FIXME!!! (just here to avoid division by 0)
        y = p.tolist()[0][1] / (w+0.0001) # FIXMEEEEE !!!!!
        #z = p.tolist()[0][2]
        return((x,y))

In [ ]:
vertices = [(-50,-50, -50), (-50,50, -50), (50,50, -50), (50,-50, -50),
            (-50,-50,  50), (-50,50,  50), (50,50,  50), (50,-50,  50)]

faces = [(4,5,6,7), (7,6,2,3), (3,2,1,0), (0,1,5,4), (0,4,7,3), (5,1,2,6)]
colors = [(255,255,0),(255,0,0),(0,255,0),(0,255,255),(0,0,255),(255,0,255)]

In [ ]:
def render(xpos: int, ypos: int, zpos: int, xrot: int = 0):
        draw.rectangle([(0,0), (300,300)], fill=0)
        for it, face in enumerate(faces):
            points = []
            for vertex_id in face:
                points.append(vertices[vertex_id])
            new_points = []
            for point in points:
                # transform in 3D             
                point = list(point)
                point.append(1)
                R1 = rotate('x', xrot)
                T1 = translate(0, 0, -100) 
                T2 = translate(xpos, ypos, zpos)
                p = T2 @ T1 @ R1 @ point
                x = p.tolist()[0][0]
                y = p.tolist()[0][1]
                z = p.tolist()[0][2]
                p = (x,y,z)
                # project to 2D
                p = project_perspective(100, p)
                #p = project_ortho(p)
                p = (p[0] + WIDTH//2, p[1] + HEIGHT//2) # move the origin to the center of the canvas
                new_points.append(p)
            # drawing filled polygons will look strange if you don't sort by depth first → Painter's Algorithm
            #draw.polygon(new_points, fill=colors[it], width=3)
            draw.polygon(new_points, outline=colors[it], width=3)
        d.update(i)
        #d.display(i) # works in PyCharm

In [ ]:
d = DisplayHandle()
d.display(i)

In [ ]:
_ = interact(render, xpos=(-100,100), ypos=(-100,100), zpos=(-20,20.0), xrot=(-180,180))

In [ ]:
# animate!
def animate():
    xrot = 0
    while True:
        xrot +=1
        render(0,0,0, xrot)
        time.sleep(0.05)

animate()

## Next steps
- add scale functions
- implement backface culling by calculating the normal of each polygon and determining whether it faces towards the viewport or away from it. 
- implement the painters algorithm: determine the center of each polygon and then sort the polygon's by Z value before drawing them
- add a camera and apply it's perspective by multiplying the inverse of the camera transform matrix onto each vertex